In [18]:
import glob
import os

import numpy as np
import pandas as pd
from skimage import io
from skimage.measure import label, regionprops
from tqdm import tqdm

env: JOBLIB_TEMP_FOLDER="/tmp"


## Extract voxel-based features from nuclear masks

Citation:
Kalinin, A.A., Hou, X., Ade, A.S., Fon, G.V., Meixner, W., Higgins, G.A., Sexton, J.Z., Wan, X., Dinov, I.D., O’Meara, M.J. and Athey, B.D. 2021. Valproic acid-induced changes of 4D nuclear morphology in astrocyte cells. _Molecular Biology of the Cell_, 32(18), pp.1624-1633. [doi:10.1091/mbc.E20-08-0502](https://doi.org/10.1091/mbc.E20-08-0502)

In [16]:
def get_feature_names():
    return [
        "area",
        "bbox_area",
        "convex_area",
        "equivalent_diameter",
        "euler_number",
        "extent",
        "inertia_tensor_eigvals",
        "major_axis_length",
        "minor_axis_length",
        "perimeter",
        "solidity",
    ]


def extract_masked_image_features(
    fname: str, dim: str = "3d", verbose: bool = False
) -> list:
    verboseprint = print if verbose else lambda *a, **k: None
    feature_names = get_feature_names()
    img = io.imread(fname)
    verboseprint(f"Img dtype: {img.dtype}, shape: {img.shape}")

    label_image, n_components = label(img, return_num=True)
    verboseprint(f"Num of components: {n_components}")

    feats = []
    for i in range(n_components):
        mask = (label_image == i + 1).astype(np.uint8)
        verboseprint(f"Mask dtype: {mask.dtype}, new shape: {mask.shape}")

        if dim.lower() == "2d":
            mask = np.max(mask, 0)
            verboseprint(f"Mask dtype: {mask.dtype}, new shape: {mask.shape}")

        props = regionprops(mask)
        del mask
        mask_feats = []
        if len(props) > 1:
            verboseprint("\nComponents:", len(props))
        else:
            for feat in feature_names:
                try:
                    mask_feats.append(props[0][feat])
                    verboseprint(f"{dim} feature: {feat}, values: {props[0][feat]}")
                except Exception:
                    verboseprint(f"No feature: {feat}")
                    pass
        feats.append(
            [y for x in mask_feats for y in (x if hasattr(x, "__iter__") else (x,))]
        )

    return np.asarray(feats)

### compute voxel features

In [8]:
def build_voxel_features_directory(
    in_path: str, out_path: str, suffix="", dim: str = "3d"
):
    dim = dim.lower()
    print("\n Folder:", in_path)
    files = sorted([f for f in glob.glob(in_path + "/" + suffix + "*.tif")])
    filenames = [os.path.split(f)[-1][:-4] for f in files]
    print("Files:", len(files))

    features = []
    mask_names = []
    for i, fname in enumerate(tqdm(files)):
        img_features = extract_masked_image_features(fname, dim=dim)
        features.extend(img_features)
        mask_names.extend(
            [
                filenames[i] + f"_mask_{j:03d}"
                for j, mask_features in enumerate(img_features)
            ]
        )

    feature_names = get_feature_names()
    if dim == "2d":
        df_columns = (
            feature_names[:6]
            + ["inertia_tensor_eigval_1", "inertia_tensor_eigval_2"]
            + feature_names[7:]
        )
    elif dim == "3d":
        df_columns = (
            feature_names[:6]
            + [
                "inertia_tensor_eigval_1",
                "inertia_tensor_eigval_2",
                "inertia_tensor_eigval_3",
            ]
            + feature_names[7:-2]
            + [feature_names[-1]]
        )

    df = pd.DataFrame(features, index=mask_names, columns=df_columns)
    df.index.name = "File"
    out_file = os.path.split(in_path)[-1]
    df.to_csv(f"{os.path.join(out_path, out_file)}{suffix}_{dim}.csv")

In [84]:
build_voxel_features_directory(
    "../../storage/data/astr_vpa/astr_vpa_postsegm", "decon/morphometry/", dim="2d"
)

  0%|          | 0/79 [00:00<?, ?it/s]


 Folder: ../../storage/data/astr_vpa/astr_vpa_postsegm
Files: 79


100%|██████████| 79/79 [01:11<00:00,  1.10it/s]


In [ ]:
build_voxel_features_directory(
    "../../storage/data/astr_vpa/astr_vpa_postsegm", "decon/morphometry/", dim="3d"
)

  0%|          | 0/79 [00:00<?, ?it/s]


 Folder: ../../storage/data/astr_vpa/astr_vpa_postsegm
Files: 79


 77%|███████▋  | 61/79 [46:14<10:13, 34.11s/it]  

In [9]:
build_voxel_features_directory(
    "../../storage/data/astr_vpa/astr_vpa_postsegm",
    "decon/morphometry/",
    suffix="12",
    dim="3d",
)

  0%|          | 0/66 [00:00<?, ?it/s]


 Folder: ../../storage/data/astr_vpa/astr_vpa_postsegm
Files: 66


100%|██████████| 66/66 [49:45<00:00, 45.23s/it]


In [10]:
build_voxel_features_directory(
    "../../storage/data/astr_vpa/astr_vpa_postsegm",
    "decon/morphometry/",
    suffix="Psi",
    dim="3d",
)

  0%|          | 0/36 [00:00<?, ?it/s]


 Folder: ../../storage/data/astr_vpa/astr_vpa_postsegm
Files: 36


100%|██████████| 36/36 [25:08<00:00, 41.89s/it]


In [11]:
build_voxel_features_directory(
    "../../storage/data/astr_vpa/astr_vpa_postsegm",
    "decon/morphometry/",
    suffix="Rho",
    dim="3d",
)

  0%|          | 0/36 [00:00<?, ?it/s]


 Folder: ../../storage/data/astr_vpa/astr_vpa_postsegm
Files: 36


100%|██████████| 36/36 [37:32<00:00, 62.57s/it]
